In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split

# one hot encoding
# check if we have cyclical values
# Check scaling
# Check features that are correlated
# Pycaret for choosing the best models, pick top 5 and then tune their hyperparameters. 

# hyperparameter-tuning: random? bayesian optimisation? grid-search takes a lot of time but is precise
    # Optuna
    # Hyper-opt

df = pd.read_csv("diabetes_data.csv")
data = df.drop("Diabetes", axis=1)
target = df["Diabetes"]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)
